In [8]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
from ipywidgets import (
    Button, FloatSlider, IntSlider, VBox, HBox, Output,
    Layout, HTML, ToggleButton, Label
)

# --------------------------- (Optional) --------------------------- #
# Uncomment the next line in Jupyter for interactive 3‑D rotation.
# %matplotlib widget

# --------------------------- Data generation ---------------------------- #

def generate_data() -> None:
    """Generate a simple linear data‑set with Gaussian noise."""
    global slope, intercept, noise_std, X, y, X_b
    slope = np.random.uniform(-5, 5)
    intercept = np.random.uniform(-10, 10)
    noise_std = np.random.uniform(0.5, 2.0)
    X = 6 * np.random.rand(100, 1) - 3
    y = intercept + slope * X + np.random.normal(0, noise_std, size=(100, 1))
    X_b = np.c_[np.ones((100, 1)), X]


generate_data()

# ----------------------------- Globals ---------------------------------- #

x_min, x_max = -3, 3
y_min, y_max = -20, 20

theta = np.zeros((2, 1))
output = Output()
step_info = HTML()
step_counter = 0

# Descent trace & gradient history
loss_history: list[float] = []
intercept_history: list[float] = []
slope_history: list[float] = []
last_gradients: np.ndarray | None = None  # for arrow drawing

# Loss‑surface cache
_theta0_vals: np.ndarray | None = None
_theta1_vals: np.ndarray | None = None
_loss_surface: np.ndarray | None = None

# -------------------- Helper: compute mean‑squared error ----------------- #

def compute_loss(th_: np.ndarray | None = None) -> float:
    th = theta if th_ is None else th_
    preds = X_b.dot(th)
    return float(np.mean((preds - y) ** 2))


# --------------------- Compute the loss surface ------------------------- #

def compute_loss_surface() -> None:
    """Pre‑compute MSE on a meshgrid for speedy re‑renders."""
    global _theta0_vals, _theta1_vals, _loss_surface

    _theta0_vals = np.linspace(-10, 10, 60)
    _theta1_vals = np.linspace(-5, 5, 60)
    T0, T1 = np.meshgrid(_theta0_vals, _theta1_vals)
    flat = np.c_[T0.ravel(), T1.ravel()]  # (N, 2)
    preds = X_b @ flat.T  # (100, N)
    mse = np.mean((preds - y) ** 2, axis=0)
    _loss_surface = mse.reshape(T0.shape)


# --------------------------- Plot helper -------------------------------- #

def _draw_gradient_arrows(ax_3d) -> None:
    """Visualise gradient components & combined vector at the current point."""
    if last_gradients is None:
        return
    gx, gy = float(last_gradients[0, 0]), float(last_gradients[1, 0])
    if gx == 0 and gy == 0:
        return
    x0, y0, z0 = theta[0, 0], theta[1, 0], loss_history[-1]
    scale = 0.4  # adjust for arrow length
    # Intercept component (green)
    ax_3d.quiver(x0, y0, z0, -gx*scale, 0, 0,
                 color="tab:green", linewidth=2, arrow_length_ratio=0.15)
    # Slope component (blue)
    ax_3d.quiver(x0, y0, z0, 0, -gy*scale, 0,
                 color="tab:blue", linewidth=2, arrow_length_ratio=0.15)
    # Combined vector (red)
    ax_3d.quiver(x0, y0, z0, -gx*scale, -gy*scale, 0,
                 color="tab:red", linewidth=2, arrow_length_ratio=0.15)


def plot_current_state() -> None:
    """Render 2‑D scatter/fit and 3‑D loss surface with path & arrows."""
    with output:
        output.clear_output(wait=True)

        fig = plt.figure(figsize=(12, 6))
        ax_data = fig.add_subplot(1, 2, 1)
        ax_3d = fig.add_subplot(1, 2, 2, projection="3d")

        # ----- Left: data and regression line ----- #
        ax_data.scatter(X, y, color="tab:blue", label="Data")
        x_line = np.linspace(x_min, x_max, 100)
        ax_data.plot(x_line, theta[0, 0] + theta[1, 0]*x_line,
                      color="tab:red", label="Fit")
        ax_data.set(xlabel="x", ylabel="y", xlim=(x_min, x_max), ylim=(y_min, y_max),
                     title="Linear regression fit")
        ax_data.legend(loc="upper left")

        # ----- Right: 3‑D loss surface ----- #
        T0, T1 = np.meshgrid(_theta0_vals, _theta1_vals)
        ax_3d.plot_surface(T0, T1, _loss_surface, cmap="viridis", alpha=0.6,
                            linewidth=0, antialiased=False)
        if loss_history:
            ax_3d.plot(intercept_history, slope_history, loss_history,
                        color="tab:red", marker="o", markersize=4, linewidth=2)
        _draw_gradient_arrows(ax_3d)
        ax_3d.set(xlabel="Intercept", ylabel="Slope", zlabel="Loss (MSE)",
                   title="Loss surface & descent path")
        ax_3d.view_init(35, -65)
        ax_3d.set_zlim(0, np.max(_loss_surface))

        plt.tight_layout()
        plt.show()


# ---------------------------- UI widgets -------------------------------- #

title_html = HTML("<h2>Regression Gradient Descent Visualizer</h2>")

step_button = Button(description="Step", button_style="success",
                     layout=Layout(width="80px"))
reset_button = Button(description="Reset", layout=Layout(width="80px"))
refresh_button = Button(description="New Data", layout=Layout(width="80px"))

lr_slider = FloatSlider(value=0.1, min=0.001, max=0.5, step=0.001,
                        description="Learning rate", readout_format=".3f",
                        layout=Layout(width="220px"))

batch_slider = IntSlider(value=100, min=1, max=100, step=5,
                         description="Batch size", layout=Layout(width="220px"))

random_init_toggle = ToggleButton(value=False, description="Random Init: OFF",
                                  layout=Layout(width="140px"))

# --------------------------- Reset logic -------------------------------- #

def reset_model() -> None:
    global theta, step_counter, loss_history, intercept_history, slope_history, last_gradients
    theta = (np.random.uniform([-10, -5], [10, 5]).reshape(2, 1)
             if random_init_toggle.value else np.zeros((2, 1)))
    step_counter = 0
    loss_history = [compute_loss()]
    intercept_history = [theta[0, 0]]
    slope_history = [theta[1, 0]]
    last_gradients = None
    compute_loss_surface()
    plot_current_state()
    step_info.value = (
        f"<b>Step:</b> {step_counter}<br>"
        f"<b>Intercept:</b> {theta[0,0]:.4f}&nbsp;&nbsp;"
        f"<b>Slope:</b> {theta[1,0]:.4f}<br>"
        f"<b>Loss:</b> {loss_history[-1]:.4f}"
    )


# -------------------------- Refresh data -------------------------------- #

def refresh_data(_):
    generate_data()
    reset_model()


# ---------------------------- Step logic -------------------------------- #

def on_step_clicked(_):
    global theta, step_counter, last_gradients

    # Mini‑batch sampling
    batch_idx = np.random.choice(len(X_b), batch_slider.value, replace=False)
    X_batch, y_batch = X_b[batch_idx], y[batch_idx]

    # Gradient computation & update
    gradients = (2 / batch_slider.value) * X_batch.T @ (X_batch @ theta - y_batch)
    theta -= lr_slider.value * gradients
    last_gradients = gradients.copy()

    # Book‑keeping
    step_counter += 1
    loss_history.append(compute_loss())
    intercept_history.append(theta[0, 0])
    slope_history.append(theta[1, 0])

    # Refresh visuals & stats
    plot_current_state()
    step_info.value = (
        f"<b>Step:</b> {step_counter}<br>"
        f"<b>Intercept:</b> {theta[0,0]:.4f}&nbsp;&nbsp;"
        f"<b>Slope:</b> {theta[1,0]:.4f}<br>"
        f"<b>Loss:</b> {loss_history[-1]:.4f}"
    )


# ----------------------- Widget interactions --------------------------- #

step_button.on_click(on_step_clicked)
reset_button.on_click(lambda _: reset_model())
refresh_button.on_click(refresh_data)


def _toggle_handler(change):
    random_init_toggle.description = "Random Init: ON" if change['new'] else "Random Init: OFF"
    reset_model()

random_init_toggle.observe(_toggle_handler, names="value")

# --------------------------- Layout ------------------------------------- #

controls = VBox([
    HBox([step_button, reset_button, refresh_button], layout=Layout(gap="8px")),
    lr_slider,
    batch_slider,
    random_init_toggle
], layout=Layout(gap="10px"))

step_info.layout = Layout(min_width="220px")

top = HBox([controls, step_info], layout=Layout(gap="40px", align_items="flex-start"))

ui = VBox([title_html, top, output],
          layout=Layout(padding="10px", border="1px solid lightgray"))

# ----------------------- Initial display -------------------------------- #
reset_model()
ui  # Display in Jupyter notebook / Voila
